In [3]:
import os

In [4]:
def read_data(filename):
    with open(filename, 'r', encoding='cp949') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

data = read_data(r'./data/감정샘플.txt')
data_text = [line[1] for line in data]
data_senti = [line[2] for line in data] # 이 값이 1이면 긍정

2. 데이터를 test와 train으로 분류

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
train_input, test_input, train_target, test_target = train_test_split(data_text, 
                                                    data_senti,
                                                   stratify = data_senti,
                                                   test_size=0.3,
                                                    random_state=158)

In [5]:
from collections import Counter

In [6]:
train_target_freq = Counter(train_target)
print('train_target_freq =', train_target_freq)
test_target_treq = Counter(test_target)
print('test_target_treq = ', test_target_treq)

train_target_freq = Counter({'1': 69158, '0': 69120})
test_target_treq =  Counter({'1': 29639, '0': 29624})


3. 피처 벡터화를 진행합니다.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
# CountVectorizer
vect = CountVectorizer(min_df = 5).fit(train_input)
X_train = vect.transform(train_input)

feature_names = vect.get_feature_names()
print('특성 개수 :', len(feature_names))
print('처음 20개 특성 :\n', feature_names[:20])

특성 개수 : 11197
처음 20개 특성 :
 ['10점', '1빠', 'ai', 'cgv', 'ebs', 'jtbc', 'kbs', 'la', 'mb', 'mbc', 'naver', 'new', 'ok', 'sbs', 'sns', 'tv', 'usb', 'ㄴㄴ', 'ㄷㄷ', 'ㅂㄷㅂㄷ']


C:\Users\admin\Anaconda3\envs\nlp\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [32]:
X_train

<138278x11197 sparse matrix of type '<class 'numpy.int64'>'
	with 767646 stored elements in Compressed Sparse Row format>

In [10]:
# Tfid 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df = 5, ngram_range=(1,2)).fit(train_input)
X_train = vect.transform(train_input)

4. 벡터화 데이터를 회귀분석을 통해 정확도 검증

- 회귀분석 이용. 교차검증 위해 cross_val_score 시행 하고 GridSearch로 여러 하이퍼파라미터로 실행

In [11]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [12]:
y_train = pd.Series(train_target)
scores = cross_val_score(LogisticRegression(solver='liblinear'), X_train, y_train, cv=5)
print('교차 검증 점수 :', scores)
print('교차 검증 점수 평균 :', scores.mean())

from sklearn.model_selection import GridSearchCV
param_grid = {'C' : [0.01, 0.1 ,1, 3, 5]}
grid = GridSearchCV(LogisticRegression(solver='liblinear'), param_grid, cv=5)
grid.fit(X_train, y_train)
print('최고 교차 검증 점수 :', round(grid.best_score_, 3))
print('최적의 매개변수 :', grid.best_params_)

교차 검증 점수 : [0.81949667 0.81884582 0.82021984 0.81894775 0.81645272]
교차 검증 점수 평균 : 0.8187925614728817
최고 교차 검증 점수 : 0.819
최적의 매개변수 : {'C': 1}


In [33]:
y_train

0         1
1         1
2         0
3         0
4         0
         ..
138273    0
138274    0
138275    0
138276    1
138277    0
Length: 138278, dtype: object

5. Test데이터를 검증 모델에 넣어 정확도를 확인해 봅니다. 
- 반드시 유의해야 할 점은 테스트 데이터에서 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer객체를 이용해 테스트 데이터를 변환(transform) 해야 한다는 것입니다. 그래야만 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아집니다. 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 CountVectorizer객체 변수인 cnt_vect.traform()을 이용해 반환합니다.

In [13]:
X_test = vect.transform(test_input)
y_test = pd.Series(test_target)
print('테스트 데이터 점수 :', grid.score(X_test, y_test))

테스트 데이터 점수 : 0.8171877900207549


6. 신규 데이터 넣어보기

In [14]:
import rhinoMorph

In [37]:
rn = rhinoMorph.startRhino()
print('rn\n', rn)
new_input = '오늘은 정말 재미있는 하루구나!'

filepath:  C:\Users\admin\Anaconda3\envs\nlp\Lib\site-packages
classpath:  C:\Users\admin\Anaconda3\envs\nlp\Lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
rn
 <java class 'rhino.RHINO'>


In [51]:
# 입력데이터 형태소 분석하기
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG','NNP','VV','VA','ZR',
                                                              'IC','MM','MAG','MAJ'])
morphed_input = ' '.join(morphed_input)  # 한개의 문자열로 만들기
inputdata.append(morphed_input)          # 분석 결과를 리스트로 만들기
X_input = vect.transform(inputdata)
print(float(grid.predict(X_input)))
result = grid.predict(X_input) # 0은 부정, 1은 긍정
print(result)
print(type(result))
if result == '0':
    print('부정적인 글입니다.')
else:
    print('긍정적인 글입니다.')

1.0
['1']
<class 'numpy.ndarray'>
긍정적인 글입니다.


In [53]:
# 한번에 모아서 실행
import rhinoMorph  

rn = rhinoMorph.startRhino()
print('rn\n', rn)
new_input = '나 오늘 행복해'

inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG','NNP','VV','VA','ZR',
                                                              'IC','MM','MAG','MAJ'])
morphed_input = ' '.join(morphed_input)  # 한개의 문자열로 만들기
inputdata.append(morphed_input)          # 분석 결과를 리스트로 만들기
X_input = vect.transform(inputdata)
print(float(grid.predict(X_input)))
result = grid.predict(X_input) # 0은 부정, 1은 긍정
print(result)
print(type(result))
if result == '0':
    print('부정적인 글입니다.')
else:
    print('긍정적인 글입니다.')

filepath:  C:\Users\admin\Anaconda3\envs\nlp\Lib\site-packages
classpath:  C:\Users\admin\Anaconda3\envs\nlp\Lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
rn
 <java class 'rhino.RHINO'>
1.0
['1']
<class 'numpy.ndarray'>
긍정적인 글입니다.
